# Pinecone 101 Webinar
In this notebook, we will learn how to create and index, embed data into vectors, load the vectors, and execute queries.

## Install Libraries

In [4]:
!pip install -qU pinecone-client==3.2.1 pinecone-notebooks tqdm



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-pinecone 0.1.1 requires pinecone-client<4.0.0,>=3.2.2, but you have pinecone-client 3.2.1 which is incompatible.
pinecone-datasets 0.7.0 requires pyarrow<12.0.0,>=11.0.0, but you have pyarrow 14.0.1 which is incompatible.
pinecone-datasets 0.7.0 requires pydantic<2.0.0,>=1.10.5, but you have pydantic 2.0 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [2]:
from pinecone_notebooks.colab import Authenticate
Authenticate()

import os
import time
import pandas as pd
from google.colab import files
from pinecone import Pinecone, ServerlessSpec
from transformers import AutoTokenizer, AutoModel
import torch

ImportError: cannot import name 'Authenticate' from 'pinecone_notebooks.colab' (/Users/rick/Library/Python/3.9/lib/python/site-packages/pinecone_notebooks/colab/__init__.py)

## Initialize Pinecone Connection

In [ ]:
# Initialize connection to Pinecone (get API key at app.pinecone.io)
api_key = os.getenv('PINECONE_API_KEY')

# Configure Pinecone client
pc = Pinecone(api_key=api_key, source_tag='pinecone-notebooks:pinecone-101')

# Get cloud and region settings
cloud = os.getenv('PINECONE_CLOUD', 'aws')
region = os.getenv('PINECONE_REGION', 'us-east-1')

# Define serverless specifications
spec = ServerlessSpec(cloud=cloud, region=region)

# Define index name
index_name = 'pinecone101'

## Create Index

In [ ]:
# Check existing indexes
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    # Create index if it does not exist
    pc.create_index(index_name, dimension=384, metric='cosine', spec=spec)
    # Wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)
    print(f"Index {index_name} has been successfully created.")
else:
    print(f"Index {index_name} already exists.")

Index pinecone101 already exists.


## Get Data
The data is based on simulated doctor's notes for numerous patients.

### Upload and Read Data

In [ ]:
# Step 1: Upload the file from your local machine to Google Colab
uploaded = files.upload()

# Step 2: Assuming the file is uploaded, read it into a DataFrame
file_name = next(iter(uploaded.keys()))
df = pd.read_json(file_name, orient='records', lines=True)

# Show head of the DataFrame
df.head()

### Connect to Index and Upload Data

In [ ]:
# Connect to index
index = pc.Index(index_name)
time.sleep(1)

# View index stats
index.describe_index_stats()

# Upsert data into index from DataFrame
index.upsert_from_dataframe(df)

## Embedding and Querying

### Define Embedding Function

In [ ]:
def get_embedding(input_question):
    model_name = 'sentence-transformers/all-MiniLM-L6-v2'
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    encoded_input = tokenizer(input_question, padding=True, truncation=True, return_tensors='pt')

    with torch.no_grad():
        model_output = model(**encoded_input)

    embedding = model_output.last_hidden_state[0].mean(dim=0)
    return embedding

### Build and Execute Query

In [ ]:
# Build a query to search
question = "what if my patient has knee pain"
query = get_embedding(question).tolist()

# Get results
results = index.query(vector=[query], top_k=3, include_metadata=True)

# Sort results by score in descending order
sorted_matches = sorted(results['matches'], key=lambda x: x['score'], reverse=True)

# Print results
print(f'Original question: {question}')
print('---\nResults:\n--------------')
for match in sorted_matches:
    print(f'ID: {match["id"]}')
    print(f'Score: {match["score"]}')
    print(f'Metadata: {match["metadata"]}')
    print('---')

Original question: what if my patient has knee pain
---
Results:
--------------
